# 來聊聊「歷史回測」吧!
<img src="fig/performance.bmp" height=430>
* reference: http://www.thertrader.com/2013/10/18/trading-strategies-performance-reporting-with-r-and-knitr/

# 投資界中的 Test-Driven Development: 歷史回測

* 使用歷史價格來驗證投資策略的表現
* 也就是說，先拿歷史資料試試水溫，確定這個策略可以挺過以前的大風大浪

# 通常回測關注哪些點？
* 像是勝率、年化報酬率、獲利風險比、持有時間、交易次數
* [R] <a href="http://www.rinfinance.com/RinFinance2009/presentations/PA%20Workshop%20Chi%20RFinance%202009-04.pdf">performanceAnalytics</a>
* <a href="http://www.thertrader.com/2013/10/18/trading-strategies-performance-reporting-with-r-and-knitr/">自動產生回測報表</a>： R & Knitr
<img src="fig/performance.bmp" height=860>

## 自從有了歷史回測，考試都考100分（？）

# 歷史回測的限制
    1. 歷史回測相信「歷史會不斷重現」，過去績效好不代表未來績效也會好
    2. 無法捕捉交互作用，例如：market impact，想想看，如果你今天買進4萬張台積電，會不會影響台積電價格？

# open source 回測工具
* [Python] <a href="https://www.quantopian.com">quantopian</a>
<img src="fig/quantopian.png" height=430>
* reference: https://www.quantopian.com

# open source 回測工具
* [R] quantmod / quantstrat
<img src="fig/quantmod.png" height=430>
* reference: 牧清華「<a href="http://www.bituzi.com/2014/12/Rbacktest6mins.html">第一次使用R語言做回測：六分鐘，就上手!</a>」

In [16]:
library(quantmod)

In [18]:
# get TWSE Index pricetwse_close <- Cl(TWII)
getSymbols('^TWII')
chartSeries(TWII)

[1] "TWII"

<img src="fig/twse.png" height=430>

In [19]:
# 取出收盤價
twse_close <- Cl(TWII)
twse_close

           TWII.Close
2007-01-02    7920.80
2007-01-03    7917.30
2007-01-04    7934.51
2007-01-05    7835.57
2007-01-08    7736.71
2007-01-09    7790.01
2007-01-10    7698.52
2007-01-11    7618.55
2007-01-12    7761.71
2007-01-15    7783.50
2007-01-16    7792.08
2007-01-17    7833.98
2007-01-18    7895.18
2007-01-19    7840.08
2007-01-22    7842.47
2007-01-23    7852.36
2007-01-24    7935.54
2007-01-25    7923.77
2007-01-26    7821.32
2007-01-29    7751.79
2007-01-30    7739.91
2007-01-31    7699.64
2007-02-01    7701.54
2007-02-02    7777.03
2007-02-05    7783.12
2007-02-06    7875.75
2007-02-07    7850.06
2007-02-08    7842.22
2007-02-09    7859.53
2007-02-12    7776.36
2007-02-13    7736.83
2007-02-14    7809.45
2007-02-26    7900.20
2007-02-27    7901.96
2007-03-01    7678.67
2007-03-02    7670.77
2007-03-05    7344.56
2007-03-06    7451.06
2007-03-07    7480.89
2007-03-08    7573.87
2007-03-09    7568.20
2007-03-12    7629.15
2007-03-13    7684.00
2007-03-14    7570.27
2007-03-15

In [20]:
# 產生交易訊號：
# 進場訊號 : 短天期 > 長天期 
# 退場訊號： 短天期 <= 長天期 

ma_short <-runMean(twse_close, n=20)  
addTA(ma_short, on=1, col="blue")

ma_long <-runMean(twse_close, n=60)  
addTA(ma_long, on=1, col="red")

<img src="fig/add_signal.png" height=430>

In [21]:
# 小心 looking forward bias ！！！！ 訊號要延遲
signal <-Lag(ifelse(ma_short>ma_long, 1,0))
signal

           Lag.1
2007-01-02    NA
2007-01-03    NA
2007-01-04    NA
2007-01-05    NA
2007-01-08    NA
2007-01-09    NA
2007-01-10    NA
2007-01-11    NA
2007-01-12    NA
2007-01-15    NA
2007-01-16    NA
2007-01-17    NA
2007-01-18    NA
2007-01-19    NA
2007-01-22    NA
2007-01-23    NA
2007-01-24    NA
2007-01-25    NA
2007-01-26    NA
2007-01-29    NA
2007-01-30    NA
2007-01-31    NA
2007-02-01    NA
2007-02-02    NA
2007-02-05    NA
2007-02-06    NA
2007-02-07    NA
2007-02-08    NA
2007-02-09    NA
2007-02-12    NA
2007-02-13    NA
2007-02-14    NA
2007-02-26    NA
2007-02-27    NA
2007-03-01    NA
2007-03-02    NA
2007-03-05    NA
2007-03-06    NA
2007-03-07    NA
2007-03-08    NA
2007-03-09    NA
2007-03-12    NA
2007-03-13    NA
2007-03-14    NA
2007-03-15    NA
2007-03-16    NA
2007-03-19    NA
2007-03-20    NA
2007-03-21    NA
2007-03-22    NA
2007-03-23    NA
2007-03-26    NA
2007-03-27    NA
2007-03-28    NA
2007-03-29    NA
2007-03-30    NA
2007-04-02    NA
2007-04-03    

In [22]:
# 產生交易損益
rtn_benchmark <- na.omit(ROC(twse_close))
rtn <- (rtn_benchmark * signal)['2008-01-01/2014-12-31']
nav <- cumprod(1+rtn)
plot(nav)

<img src="fig/pnl.png" height=430>

In [23]:
## 暴搜參數範圍
get.signal <- function(day_shorter=20, day_longer=60){
  ma_shorter <-runMean(twse_close, n=day_shorter)
  ma_longer <-runMean(twse_close, n=day_longer) 
  signal <-Lag(ifelse(ma_shorter > ma_longer, 1, 0))
  signal
}

get.backtest.rtn <- function(signal, rtn_benchmark){
  rtn <- (rtn_benchmark * signal)['2008-01-01/2014-12-31']
  rtn
}

In [28]:
# 暴搜參數範圍
days_shorter <- c(10, 15, 20, 25, 30)
days_longer <- c(40, 45, 50, 55, 60, 65)

result <- data.frame(idx=1:1733)
for (day_shorter in days_shorter){
  for(day_longer in days_longer){
    signal <- get.signal(day_shorter, day_longer)
    rtn_benchmark <- ROC(twse_close)
    rtn <- get.backtest.rtn(signal, rtn_benchmark)
    result[paste(day_shorter, day_longer, sep=',')] <- rtn
  }
}
result <- result[, -1]
write.csv(result, '~/coscup_pbo/data/backtest_result.csv', row.names=F)

In [30]:
# 得到不同策略的每日損益
f <- read.csv('~/coscup_pbo/data/backtest_result.csv', header=T)
tail(df)

,X10.40,X10.45,X10.50,X10.55,X10.60,X10.65,X15.40,X15.45,X15.50,X15.55,X15.60,X15.65,X20.40,X20.45,X20.50,X20.55,X20.60,X20.65,X25.40,X25.45,X25.50,X25.55,X25.60,X25.65,X30.40,X30.45,X30.50,X30.55,X30.60,X30.65
1728,0,0.0002979172,0.0002979172,0.0002979172,0.0002979172,0.0002979172,0.0002979172,0.0002979172,0.0002979172,0.0002979172,0.0002979172,0.0002979172,0.0002979172,0.0002979172,0.0002979172,0.0002979172,0.0002979172,0.0002979172,0.0002979172,0.0002979172,0.0002979172,0.0002979172,0.0002979172,0.0002979172,0.0002979172,0.0002979172,0.0002979172,0.0002979172,0.0002979172,0.0002979172
1729,0,0,0.009677418,0.009677418,0.009677418,0.009677418,0.009677418,0.009677418,0.009677418,0.009677418,0.009677418,0.009677418,0.009677418,0.009677418,0.009677418,0.009677418,0.009677418,0.009677418,0.009677418,0.009677418,0.009677418,0.009677418,0.009677418,0.009677418,0.009677418,0.009677418,0.009677418,0.009677418,0.009677418,0.009677418
1730,0,0.003031551,0.003031551,0.003031551,0.003031551,0.003031551,0.003031551,0.003031551,0.003031551,0.003031551,0.003031551,0.003031551,0.003031551,0.003031551,0.003031551,0.003031551,0.003031551,0.003031551,0.003031551,0.003031551,0.003031551,0.003031551,0.003031551,0.003031551,0.003031551,0.003031551,0.003031551,0.003031551,0.003031551,0.003031551
1731,0,0.007806374,0.007806374,0.007806374,0.007806374,0.007806374,0.007806374,0.007806374,0.007806374,0.007806374,0.007806374,0.007806374,0.007806374,0.007806374,0.007806374,0.007806374,0.007806374,0.007806374,0.007806374,0.007806374,0.007806374,0.007806374,0.007806374,0.007806374,0.007806374,0.007806374,0.007806374,0.007806374,0.007806374,0.007806374
1732,-0.001924103,-0.001924103,-0.001924103,-0.001924103,-0.001924103,-0.001924103,-0.001924103,-0.001924103,-0.001924103,-0.001924103,-0.001924103,-0.001924103,-0.001924103,-0.001924103,-0.001924103,-0.001924103,-0.001924103,-0.001924103,-0.001924103,-0.001924103,-0.001924103,-0.001924103,-0.001924103,-0.001924103,-0.001924103,-0.001924103,-0.001924103,-0.001924103,-0.001924103,-0.001924103
1733,0.004180748,0.004180748,0.004180748,0.004180748,0.004180748,0.004180748,0.004180748,0.004180748,0.004180748,0.004180748,0.004180748,0.004180748,0.004180748,0.004180748,0.004180748,0.004180748,0.004180748,0.004180748,0.004180748,0.004180748,0.004180748,0.004180748,0.004180748,0.004180748,0.004180748,0.004180748,0.004180748,0.004180748,0.004180748,0.004180748
